# Deploy a Distributed Pytorch Recommendation model in Amazon SageMaker
Previously, we trained a two tower model that produced a user and a movie embedding table. These embedding tables allows a recommendation engine to find similar movies to the user features in the embedding space. Performing similarity search across embedding space is efficient using Cosine similarity, or Dot product or using ANN. Apart from the efficiency gain from retrieving relevant movies using an optimized similarity search algorithm, this retrieval process can be used to narrow the number of potential movies from million of titles to tens or hundreds. 

The retrieval process is depicted in the following diagram:

<img src="img/two-tower-retrieval.png" width="800">

The two tower system is sometimes called a 2 stage recommendation system. 
A two-stage recommendation system consists of the following component:

* Candidate Generation (First Stage): Quickly retrieves thousands of relevant items from a massive catalog of millions or billions of items
* Ranking (Second Stage): A more powerful model that precisely ranks the retrieved candidates

The movie retrieval engine described above is addressed in the first stage: candidate generation. The second stage involves a ranking model to provides ranking of the candidate movies based on the relevance. Putting everything together, here's a complete two tower recommendation system in a pipeline:

<img src="img/two-stage-retrieval-recsys.png" width="800">

In this lab, we are going to focus on deploying a candidate retrieval model developed using [TorchRec](https://pytorch.org/torchrec/) framework. Here's an updated diagram with dotted lines that highlights the model we are going to be deploying in this lab.

<img src="img/two-stage-retrieval-recsys-retrieval-only.png" width="800">



In [ ]:
%store -r

Install sagemaker dependencies

In [ ]:
%pip install sagemaker -q -U

Import required python libraries

In [ ]:
import sagemaker
import boto3
from sagemaker.local import LocalSession
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.s3 import S3Uploader
from datetime import datetime
import numpy as np
import json

In [ ]:
# session = LocalSession()
# session.config = {'local': {'local_code': True } }
session = sagemaker.Session()
bucket = session.default_bucket()
role = get_execution_role()
model_data_url = sm_model_s3_url
region = session.boto_region_name

# Deploy Pytorch Model using TorchServe in SageMaker 
We've now ready to deploy the model for serving recommendations. SageMaker supports the following ways to deploy a model, depending on your use case:

* For persistent, real-time endpoints that make one prediction at a time, use SageMaker AI real-time hosting services. For more information, see [Real-time inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html).

* Workloads that have idle periods between traffic spikes and can tolerate cold starts, use Serverless Inference. For more informatoin, see [Deploy models with Amazon SageMaker Serverless Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints.html).

* Requests with large payload sizes up to 1GB, long processing times, and near real-time latency requirements, use Amazon SageMaker Asynchronous Inference. For more information, see [Asynchronous inference](https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference.html).

* To get predictions for an entire dataset, use SageMaker AI batch transform. For more information, see [Batch transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) for inference with Amazon SageMaker AI.

Here's a diagram that summarizes the different deployment modes described above:

<img src="img/sagemaker-deployment-modes.png" width="1000">


For our lab, we'll deploy a [Real-time inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html) for serving inferences using TorchServe with SageMaker SDK. 

**Note:** The example shown below deploys a pytorch model to an endpoint behind a single GPU instance. For advanced use cases which involves multiple GPUs or GPU instances with torchserve, please refer to [this](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-models-frameworks-torchserve.html) documentation.

## AWS Deep Learning Containers
[AWS Deep Learning Containers](https://github.com/aws/deep-learning-containers) (DLCs) are a set of Docker images for training and serving models in TensorFlow, TensorFlow 2, PyTorch and others. Deep Learning Containers provide optimized environments with TensorFlow, Nvidia CUDA (for GPU instances), and Intel MKL (for CPU instances) libraries and are available in the Amazon Elastic Container Registry (Amazon ECR).

To retrieve a specific container image, you can directly reference the ECR URI in the github link, or use SageMaker SDK to return the proper URI based on the frameowork version. 


In [ ]:
# Find inference container image
inference_image_uri = sagemaker.image_uris.retrieve(
    framework='djl-lmi', # use lmi image until deep learning image is available
    region=region,
    py_version='py311',
    image_scope="inference",
    instance_type='ml.p3.2xlarge'
)
# temporary until sdk updates djl container images fpr py311
inference_image_uri = f'763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.32.0-lmi14.0.0-cu126'

## DJL Serving

DJL Serving is a high performance universal stand-alone model serving solution. It takes a deep learning model, several models, or workflows and makes them available through an HTTP endpoint.

DJL Serving accepts the following artifacts in your archive:

- Model checkpoint: Files that store your model weights.

- serving.properties: A configuration file that you can add for each model. Place serving.properties in the same directory as your model file.

- model.py: The inference handler code. This is only applicable when using Python mode. If you don't specify model.py, djl-serving uses one of the default handlers.

The following is an example of a model.tar.gz structure:


```
- model_root_dir # root directory
  - serving.properties # A configuration file that you can add for each model. Place serving.properties in the same directory as your model file.      
  - model.py # your custom handler file for Python, if you choose not to use the default handlers provided by DJL Serving
  - model binary files # used for Java mode, or if you don't want to use option.model_id and option.s3_url for Python mode
```

For more information about DJL Serving, please refer to this [link](https://docs.djl.ai/master/docs/serving/index.html).

Download model binary files and add inference code

In [ ]:
!aws s3 cp {model_data_url} model/model.tar.gz

In [ ]:
%%sh
rm -rf temp/ && mkdir -p temp && cd temp
tar -xvzf ../model/model.tar.gz >/dev/null 2>&1
cp -R ../src/* ./ && rm -rd data && rm train.py
tar -cvzf ../model/model.tar.gz . >/dev/null 2>&1 && cd .. && rm -rf temp

In [ ]:
model_name = f"two-tower-{datetime.now():%Y-%m-%d-%H-%M-%S}"
model_url = S3Uploader.upload(
    local_path="model/model.tar.gz",
    desired_s3_uri=f"s3://{bucket}/models/{model_name}",
)

In [ ]:
# Create model
model = Model(
    image_uri=inference_image_uri,
    source_dir="src",
    model_data=model_url,
    role=role,
    sagemaker_session=session,
    predictor_cls=Predictor,
    env={
        "MASTER_ADDR" : "localhost", 
        "MASTER_PORT" : "12356", 
        "CUDA_VISIBLE_DEVICES" : "0",
        "LOCAL_RANK" : "0",
        "WORLD_SIZE" : "1"
    })

Deploy the torch model

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    # instance_type='local_gpu', # uncomment if running in a local mode.
    instance_type='ml.p3.2xlarge',
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
    wait=False
)

The following is the code for [model.py](src/model.py)

In [ ]:
%load src/model.py

Wait for the endpoint to be ready to serve requests.

In [ ]:
import boto3
import time

def wait_for_endpoint(endpoint_name, timeout_minutes=30):
    sagemaker_client = boto3.client('sagemaker')
    max_attempts = timeout_minutes * 2  # Check every 30 seconds
    attempts = 0
    
    while attempts < max_attempts:
        response = sagemaker_client.describe_endpoint(
            EndpointName=endpoint_name
        )
        status = response['EndpointStatus']
        
        if status == 'InService':
            return True
        
        if status in ['Failed', 'OutOfService', 'Deleting']:
            raise Exception(f"Endpoint failed with status: {status}")
            
        time.sleep(30)
        attempts += 1
    
    raise TimeoutError(f"Endpoint did not become ready within {timeout_minutes} minutes")

In [ ]:
wait_for_endpoint(predictor.endpoint_name)

Test the endpoint with random userIds

In [ ]:
input_data = {"inputs": [1234, 534, 30]} #userIds
response = predictor.predict(input_data)

In [ ]:
response

Saving model information for next lab

In [ ]:
model_name = predictor._get_model_names()[0]
model_serving_data_s3_uri = predictor.sagemaker_session.sagemaker_client.describe_model(
    ModelName=model_name
)['PrimaryContainer']['ModelDataUrl']

In [ ]:
%store model_serving_data_s3_uri

### Delete Endpoint


In [ ]:
predictor.delete_endpoint()

# Next Step
Congratulations! You've completed the end to end process of deploying a Pytorch model into Torchserve using SageMaker. 
In the next lab, we'll explore [Shadow testing](https://docs.aws.amazon.com/sagemaker/latest/dg/shadow-tests.html), a unique feature i SageMaker that allows you to evaluate new model variants alongside your existing production model without impacting live traffic or end users. Go ahead and open [03-sm-inference-shadow-test.ipynb](03-sm-inference-shadow-test.ipynb) and follow the instructions in the notebook.